In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/HepaticVessel_experiment


In [2]:
import pandas as pd
import numpy as np
import pydicom as dicom
import nibabel as nib
import os
from sklearn.model_selection import KFold,train_test_split
import json

/home/dmitriishubin/anaconda3/envs/thesis/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
img = nib.load('../data/Task09_Spleen/imagesTr/spleen_10.nii.gz')
img = np.array(img.dataobj)

In [4]:
img.shape

(512, 512, 55)

In [5]:
patients = [i for i in os.listdir('../data/Task09_Spleen/imagesTr/') if i.find('._sp')==-1]

In [6]:
patients

['spleen_18.nii.gz',
 'spleen_63.nii.gz',
 'spleen_16.nii.gz',
 'spleen_41.nii.gz',
 'spleen_62.nii.gz',
 'spleen_6.nii.gz',
 'spleen_47.nii.gz',
 'spleen_61.nii.gz',
 'spleen_60.nii.gz',
 'spleen_59.nii.gz',
 'spleen_22.nii.gz',
 'spleen_28.nii.gz',
 'spleen_25.nii.gz',
 'spleen_14.nii.gz',
 'spleen_3.nii.gz',
 'spleen_17.nii.gz',
 'spleen_52.nii.gz',
 'spleen_49.nii.gz',
 'spleen_56.nii.gz',
 'spleen_24.nii.gz',
 'spleen_40.nii.gz',
 'spleen_13.nii.gz',
 'spleen_45.nii.gz',
 'spleen_9.nii.gz',
 'spleen_26.nii.gz',
 'spleen_46.nii.gz',
 'spleen_38.nii.gz',
 'spleen_21.nii.gz',
 'spleen_32.nii.gz',
 'spleen_53.nii.gz',
 'spleen_12.nii.gz',
 'spleen_19.nii.gz',
 'spleen_20.nii.gz',
 'spleen_27.nii.gz',
 'spleen_29.nii.gz',
 'spleen_8.nii.gz',
 'spleen_44.nii.gz',
 'spleen_33.nii.gz',
 'spleen_31.nii.gz',
 'spleen_2.nii.gz',
 'spleen_10.nii.gz']

In [7]:
patients[0][:-7]

'spleen_18'

In [ ]:
names = []

os.makedirs('../data/Task06_Lung/processed_data',exist_ok=True)

count = 0

for patient in patients:
    img = nib.load(f'../data/Task06_Lung/imagesTr/{patient}')
    img = np.array(img.dataobj)
    seg = nib.load(f'../data/Task06_Lung/labelsTr/{patient}')
    seg = np.array(seg.dataobj)
    
    patient = patient[:-7]
    
    os.makedirs(f'../data/Task06_Lung/processed_data/{patient}',exist_ok=True)
    
    for i in range(img.shape[-1]):
        temp_img = img[:,:,i].reshape(320, 320,1)
        temp_seg = seg[:,:,i].reshape(320, 320,1)
        
        
        np.save(f'../data/Task06_Lung/processed_data/{patient}/{i}.npy',temp_img)
        np.save(f'../data/Task06_Lung/processed_data/{patient}/{i}_seg.npy',temp_seg)
        count+=1
            

In [8]:
for index,patient in enumerate(patients):
    patients[index]=patient[:-7]

In [ ]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    images_train = []
    for patient in patients_train:
        images_train+=[
                f'../data/Task06_Lung/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
    
    
    images_val = []
    for patient in patients_test:
        images_val+=[
                f'../data/Task06_Lung/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
        
    
    split = {
        'train': images_train,
        'val': images_val,
    }
    
    
    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)

In [9]:
patients

['spleen_18',
 'spleen_63',
 'spleen_16',
 'spleen_41',
 'spleen_62',
 'spleen_6',
 'spleen_47',
 'spleen_61',
 'spleen_60',
 'spleen_59',
 'spleen_22',
 'spleen_28',
 'spleen_25',
 'spleen_14',
 'spleen_3',
 'spleen_17',
 'spleen_52',
 'spleen_49',
 'spleen_56',
 'spleen_24',
 'spleen_40',
 'spleen_13',
 'spleen_45',
 'spleen_9',
 'spleen_26',
 'spleen_46',
 'spleen_38',
 'spleen_21',
 'spleen_32',
 'spleen_53',
 'spleen_12',
 'spleen_19',
 'spleen_20',
 'spleen_27',
 'spleen_29',
 'spleen_8',
 'spleen_44',
 'spleen_33',
 'spleen_31',
 'spleen_2',
 'spleen_10']

In [10]:
patients = np.array(patients)

for fold in range(5):

    patients_train, patients_test = train_test_split(patients, test_size=0.3, random_state=fold)

    patients_train, patients_val = train_test_split(patients_train, test_size=0.45, random_state=fold)

    images_train = []
    for patient in patients_train:
        images_train+=[
                f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]


    images_val = []
    for patient in patients_val:
        images_val+=[
                f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]

    images_test = []
    for patient in patients_test:
        images_test+=[
                f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]


    split = {
        'train': images_train,
        'val': images_val,
        'test': images_test,
    }


    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)

In [11]:
len(patients_train)

15

In [12]:
len(patients_val)

13

In [13]:
len(patients_test)

13

In [ ]:
f'../data/Task02_Heart/processed_data/{patients_train}/'